In [11]:
# Here I will be building out the architecture of a classification LSTM
# At each time step, this LSTM will take in a vector representing autoencoder output
# Its goal is to output whether or not the movie induces fear at each time step


# First, import necessary libraries
import tensorflow as tf
import numpy as np

In [12]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local08 import *

In [13]:
# uploading the X values from the autoencoder
X_input = np.load('./auto_output01.npy')
print(X_input.shape)

(7, 219, 12288)


In [14]:
# uploading the Y values
y_data_input = fear_oneHot(212, 'fear_annotations_part01/MEDIAEVAL18_7_Fear.txt')
print(y_data_input.shape)

(212,)


In [15]:
# partition data into training and testing sets
train_num = 4
val_num = 3

X_train_data = X_input[:train_num, :, :]
X_valid_data = X_input[train_num:train_num+val_num, :, :]

print(X_train_data.shape)
print(X_valid_data.shape)

(4, 219, 12288)
(3, 219, 12288)


In [21]:
# uploading the y training data
timesteps = X_train_data.shape[1]
data_dim = X_train_data.shape[2]

# set up y_train_data master array
Y_train_data = np.zeros([train_num, timesteps])

# for each movie number between and including 7 and 13
for num in range(train_num):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+num))
    # create a one-hot vector
    y_data = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_train_data[num, :] = y_data
print(Y_train_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]


In [19]:
# upload the Y validation set

Y_valid_data = np.zeros([val_num, timesteps])

# for each movie number in validation set
for num in range(val_num):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+ train_num + num))
    # create a one-hot vector
    y_valid = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_valid_data[num, :] = y_valid
print(Y_valid_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]


In [28]:
# constructing a many-to-many LSTM model in keras --> inspiration: https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras
# training on autoencoder output
model = Sequential()
model.add(LSTM(timesteps, return_sequences=True,
               input_shape=(timesteps, data_dim)))

# flatten --> dense combo
model.add(Flatten()) 
# add the final dense layer and then softmax
model.add(Dense(timesteps, activation='softmax'))

In [29]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy'])

In [30]:
# running the LSTM model
model.fit(X_train_data, Y_train_data, epochs = 20, validation_data=(X_valid_data, Y_valid_data))
print("finished training!")

Train on 4 samples, validate on 3 samples
Epoch 1/20
4/4 [==============================] - 12s 3s/step - loss: 1.0413 - binary_accuracy: 0.8116 - val_loss: 1.2930 - val_binary_accuracy: 0.7763
Epoch 2/20
4/4 [==============================] - 9s 2s/step - loss: 0.9625 - binary_accuracy: 0.8116 - val_loss: 3.0324 - val_binary_accuracy: 0.7763
Epoch 3/20
4/4 [==============================] - 9s 2s/step - loss: 1.7071 - binary_accuracy: 0.8116 - val_loss: 3.1896 - val_binary_accuracy: 0.7747
Epoch 4/20
4/4 [==============================] - 9s 2s/step - loss: 2.0373 - binary_accuracy: 0.8139 - val_loss: 3.3674 - val_binary_accuracy: 0.7747
Epoch 5/20
4/4 [==============================] - 9s 2s/step - loss: 2.3690 - binary_accuracy: 0.8151 - val_loss: 3.3715 - val_binary_accuracy: 0.7763
Epoch 6/20
4/4 [==============================] - 9s 2s/step - loss: 2.4924 - binary_accuracy: 0.8128 - val_loss: 3.4875 - val_binary_accuracy: 0.7763
Epoch 7/20
4/4 [==============================] - 1

In [26]:
# trying to view the model output
out = model.predict(X_train_data)
print("model prediction:")
print(out[0])
print("target:")
print(Y_train_data[0])

print("before 64:")
print(out[0][63])
print("64:")
print(out[0][65])

print("rounded")
print(np.round(out)[0])

model prediction:
[1.5196355e-07 3.9365766e-08 1.1666211e-08 2.9819137e-08 1.1119864e-07
 2.5930452e-08 3.4395995e-08 9.4654418e-09 6.4271333e-08 9.0170644e-09
 2.6179277e-08 2.1710793e-08 5.3078217e-08 8.9951104e-09 8.2388061e-09
 1.5145423e-08 4.4607084e-08 2.6558396e-07 2.3572177e-07 2.8888974e-07
 7.0404158e-06 3.7715604e-06 4.2565114e-07 4.3033810e-07 3.0542708e-07
 8.1208827e-06 9.7397096e-06 1.8496506e-06 4.9017668e-05 3.4772820e-05
 2.8764445e-05 2.4046751e-05 9.5762472e-05 1.7088316e-05 5.4705069e-05
 3.5920857e-05 6.1904109e-07 8.6675769e-05 8.4074010e-05 1.1712875e-06
 9.4848349e-07 7.0600770e-08 7.8952624e-08 5.6531437e-08 1.5574985e-07
 4.7688481e-08 9.9840307e-08 4.6553765e-08 1.2508207e-07 5.0673677e-08
 8.6592008e-06 6.0885242e-08 4.1828447e-05 1.0177346e-04 5.6338518e-05
 1.7602665e-04 1.0976451e-04 6.3210391e-05 4.8141286e-05 7.8665085e-05
 1.4424013e-07 1.2598485e-07 6.2352754e-08 2.9701752e-08 9.9999297e-01
 9.9998963e-01 9.9998355e-01 9.9997711e-01 9.9999118e-01 9.

In [ ]:
#try visualizing this model at some point?